# Sprint 13 - Análisis de Componentes Independientes (Sesiones)
**Versión para estudiantes**

Continuamos aprendiendo en este caso sobre técnicas asociadas al **álgebra lineal**. En esta oportunidad nos enfocamos en el **Análisis de Componentes Independientes (ICA)** un método similar al *PCA* en cuanto a su derivación a partir de descomposición de matrices, pero con aplicaciones distintas, aunque bastante interesantes, como lo veremos a continuación. 

## Entendimiento del contexto

La Fiscalía de tu país ha decidido contratarte a fin de que le proporciones tus conocimientos como científico de datos en la resolución de un caso de corrupción. En concreto, se desea contar con evidencia para vincular a un sospechoso, para lo cual se pretende saber si dicha persona participó de una reunión donde se discutió sobre el pago de montos extraoficiales por la asignación de contratos con el Estado (un crimen comúnmente conocido ocomo peculado). 

En esta reunión, la Fiscalía implantó 3 micrófonos ocultos que le permitieron grabar poco menos de 5 minutos de las conversaciones en una calidad moderada. En cualquier caso, si estas conversaciones son debidamente tratadas y en efecto se descubre que el sospechoso participó en la reunión, se tendrá de insumos sólidos para vincularlo penalmente y llevarlo a juicio finalmente.

Por tanto, te han solicitado que mediante tus conocimientos en análisis de datos, estadística y álgebra lineal, identifiques si efectivamente esta persona participó de la reunión.

## Entendimiento de los datos

Importa las librerías que utilizarás incluyendo las funciones `StandardScaler` y `r2_score`.

La Fiscalía ha decodificado en decibeles (dB) los audios extraídos de las 3 grabaciones y los ha incorporado en la tabla **grabaciones**, la cual contiene 2,672 filas y 3 columnas (donde cada una de ellas representa una de las grabaciones respectivamente). 

Adicional a esto, la Fiscalía también te ha compartido otra grabación en el archivo **mensaje** que contiene la información -igual en dB- de un audio con la voz del sospechoso. Esta grabación fue levantada en unas declaraciones en medios que realizó la persona hace un par de semanas, y que tiene una duración de alrededor un minuto descompuesto en 600 décimas de segundo.

Explora estos datasets a fin de conocerlos en mayor detalle.

Retomando el caso en cuestión, si el sospechoso se encontró en la reunión sería lógico suponer que su voz quedó registrada en las grabaciones, aunque mezclada entre las voces de otros participantes y diversos ruidos ambientales. El desafío está entonces en buscar una manera de aislar cada una de estas fuentes de audio, y justamente existe una técnica de álgebra lineal llamada **descomposición singular** que puede ayudarnos y que es la base matemática dentrás del *ICA*.

## Aplicación de ICA

El Ánálisis de Componentes Independientes (*ICA* por sus siglas en inglés), consiste de un proceso que combina métodos estadísticos y de álgebra lineal, a fin de extraer los componentes 
$c$ que mediante una combinación lineal generan las variables $x$ que son parte de un dataset estandarizado $X$. En otras palabras para toda variable $x$ del dataset, se cumple que 

$$ x = \sum_{c} \kappa_c c $$

donde $\kappa_c$ es un peso asociado al componente $c$. Además, como su nombre lo indica estos componentes son independientes entre sí, por lo cual su covarianza es 0.

Para tener una comprensión mayor del algoritmo *ICA*, conviene compararlo con el *PCA*. Por una parte, notemos que el segundo utiliza métodos matemáticos para combinar las variables $x$ del dataset en nuevos atributos $x_{pca}$ independientes tal que 

$$ X_{pca} = f(X) = X\cdot Q $$

En cambio, el *ICA* hace un proceso "inverso" en cuanto a que busca descomponer cada $x$ en sus componentes tal que

$$ x = f(C) = C\cdot \kappa,\quad x\in X $$

A continuación entonces, vamos a realizar paso a paso el proceso del *ICA*.

### Estandarización del dataset

Como ya conoces, el *PCA* requiere que las variables en $X$ estén centradas con media 0 solamente. El *ICA* por su parte requiere que además su desviación estandar sea 1, o lo que es lo mismo que $X$ esté estandarizado.

Estandariza por tanto tus datos de grabaciones en la reunión y guarda el resultado en la variable `X`.

### Descomposición singular para ICA

A diferencia del *PCA* que aplica una descomposición de vectores propios de la matriz de covarianza $\Sigma_X$, el *ICA* requiere de una descomposición de vectores singulares de dicha matriz tal que

$$ \Sigma_X = U\cdot \Lambda_o\cdot V $$

donde $U$ y $V$ corresponden a matrices **ortogonales**, y $\Lambda_o$ es una matriz diagonal que contiene los valores propios $\lambda$ de $\Sigma_X$, solo que en esta ocasión ordenados de mayor a menor. Vale recordar que las matrices cuadradas pueden tener más de una factorización, por lo que esta definición no es contradictoria con la descomposición vista para *PCA*.

Sigamos con unas definiciones complementarias. Sea además $d_i$ el i-ésimo valor singular de la matriz $\Sigma_X$ tal que

$$ d_i = \sqrt{|\lambda_i|} $$

Por tanto, el valor singular se define como la norma o magnitud del vector propio $v_i$ asociado.

Finalmente, se puede definir $D$ como la matriz diagonal que contiene los valores singulares ordenados de mayor a menor, y $K$ a la matriz $U$ normalizada por el valor $d_i$ correspondiente en cada columna. Entonces, se cumple que

$$ K\cdot D\cdot V = I $$

donde $I$ es la matriz identidad y como ya se explicó

$$ K = [k_1 \quad ... \quad k_n] = \left[ \frac{u_1}{d_1} \quad ... \quad \frac{u_n}{d_n} \right] $$

Visto esto, realiza manualmente esta descomposición iniciando por calcular la matriz de covarianzas de $X$.

Ejecuta a continuación la descomposición singular con la funcion `np.linalg.svd`.

Calcula las matrices $D$ y $K$ conforme la definición expuesta.

Verifica la condición planteada en las definiciones respecto a que $K\cdot D\cdot V = I$.

¿Cómo se relacionan estos cálculos con el *ICA* y con nuestro caso en particular? Pues bien, a la matriz $K$ se la conoce como **matriz de blanqueo** puesto que al proyectar los datos en el espacio correspondiente a ella (o lo que es lo mismo al multiplicar $X\cdot K$), se garantiza que exista independencia entre las variables.

Comprueba esta proposición, esto es, blanquea los datos.

### Desmezclado de componentes

Hasta este punto hemos logrado asegurar que las 3 señales de audio sean completamente autónomas, es decir, que el sonido captado por cada una no se replique en las demás. Sin embargo, aún no hemos podido distinguir las fuentes específicas que las componen.

Para conseguir esto, aplicaremos el criterio de **no - gaussianeidad** de variables. Para comprender de qué se trata, podemos asumir que si dos variables no comparten una misma distribución estadística tendrían un origen significativamente distinto. Y en nuestro contexto, es lógico suponer que las fuentes de audio diferentes, producirían señales estadísticamente distintas.

No se pretende entrar en el detalle matemático de este concepto debido a su complejidad y extensión, sin embargo el código a continuación es una implementación que consigue lo anterior mediante lo expuesto en Hyvarinen et al., 2000. Si te interesa leer este artículo te dejo el link oficial a continuación: https://www.cs.helsinki.fi/u/ahyvarin/papers/NN00new.pdf

```py
# Obtener una primera matriz W al azar
np.random.seed(123)
W = np.random.rand(K.shape[1],K.shape[1])
uw, dw, vw = np.linalg.svd(W)
W = uw @ vw

# Iterar hasta obtener W final
for i in range(5000):

    # Desmezclar X con W anterior
    XW = X_b @ W.T

    # Obtener nueva matriz W con criterio de no-gaussianeidad
    g1 = np.tanh(XW)
    g2 = 1 - g1**2
    mat1  =  X_b.T @ g1 / X_b.shape[0]
    mat2 = W.T @ np.diag(g2.mean())
    negent = mat1 - mat2
    uw, dw, vw = np.linalg.svd(negent)
    W_new = uw @ vw

    # Reasignar nuevo W
    W = W_new

print(W)
```

De una forma muy simplista, lo que buscamos hacer es encontrar una matriz normalizada $W$ conocida como **matriz de desmezclado**, con la cual se pueden derivar las señales originales $X_{ica}$, al proyectar los datos blanqueados $X_b$ en el espacio de $W^t$. Es decir

$$ X_{ica} = X_b \cdot W^t $$

Proyecta ahora los datos blanqueados en $W^t$. 

## Resolución del caso

Ya contamos con unas potenciales fuentes de sonido independientes y aisladas entre ellas gracias al *ICA* realizado. Nos queda evaluar si alguna de estas señales se corresponde con aquella del sospeshoso.

Para esto, en primera instancia "des-escala" las columnas de $X_{ica}$ a fin que sean comparables con los datos de la entrevista. En otras palabras, multiplica cada una de estas columnas por la desviación estandar de los datos de la entrevista y súmales su media.

A continuación filtra los primeros 600 registros (un minuto) de estas señales, e incorpora como una columna adicional el audio de la entrevista.

Contrasta cada una de las fuentes extraídas con los datos de la entrevista para identificar alguna coincidencia.

De los resultados alcanzados se desprende que existe evidencia significativa de que el sospechoso participó de la reunión investigada por la Fiscalía. Por tanto, la entidad ya cuenta con insumos para preparar su caso contra esta persona gracias a tus habiidades en ciencia de datos y álgebra lineal. 

## ICA con Scikit-Learn

Para terminar, vale que conozcas la función `FastICA` de **Scikit-Learn** que implementa el algoritmo visto en este caso de manera automática. Aplícalo entonces con tus datos y mira los resultados alcanzados, que son incluso mejores a los vistos en el paso a paso anterior debido a que emplea métodos de optimización avanzados.